In [1011]:
!pip install cohens_d

In [1012]:
import pandas as pd
import numpy as np
import ast
import json
from collections import Counter, defaultdict

In [1013]:
df_arguments_sentiment = pd.read_csv("sentiment_overview_3.5-turbo_v3.csv")
df_paragraphs_arguments = pd.read_csv("report_arguments_3.5_v3.csv")
df_reports = pd.read_csv("preprocessed_reports.csv")

In [1014]:
df_arguments_sentiment = df_arguments_sentiment.drop(columns="Unnamed: 0")
df_paragraphs_arguments = df_paragraphs_arguments.drop(columns=["Unnamed: 0","Unnamed: 0.1","paragraph"])
df_reports = df_reports.drop(columns=["Unnamed: 0","paragraphs"])

In [1015]:
list_of_arguments = [
    "Growth",
    "Price/Earnings Ratio",
    "Earnings per Share",
    "Cash Flow",
    "Revenue",
    "Return On Equity",
    "Margins",
    "Cost Management",
    "Dividend Policy",
    "Investments",
    "Balance Sheet",
    "Long-term Growth",
    "Mergers and Acquisition",
    "Refranchising",
    "Sustainability",
    "Employees",
    "Research and Development",
    "Marketing",
    "Shares Repurchase",
    "Processes",
    "Leadership",
    "Innovation",
    "Product Characteristics",
    "Pricing Strategy",
    "Production",
    "Technology Trends",
    "Market Share",
    "Market Conditions",
    "Market Expansion",
    "Competition",
    "Global Presence",
    "Industry Outlook",
    "Regulations",
    "Partnerships and Collaborations",
    "Supply Chain",
    "Economic Conditions",
    "Demand",
    "Customers"
]


In [1016]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Expa..."
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership..."
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Innovation': 'positive', 'Technology Trends'..."
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Technology Trends': 'positive', 'Market Shar..."
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Technology Trends': 'n..."
...,...,...,...,...
12567,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Market Share': 'positive', 'Revenue': 'negat..."
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi..."
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,{'Market Share': 'positive'}
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,{'Growth': 'positive'}


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument Aggreation</h2>
</div>

In [1017]:
# Turn string dictionary back to actual dictionary
df_paragraphs_arguments["provided_arguments"] = df_paragraphs_arguments["provided_arguments"].apply(ast.literal_eval)

In [1018]:
from collections import Counter

# Step 2: Count occurrences of 'positive', 'negative', 'neutral' in each row
def count_sentiments(arguments):
    counts = Counter(arguments.values())  # Count occurrences of each sentiment
    return {'positive_arguments': counts.get('positive', 0),
            'neutral_arguments': counts.get('neutral', 0),
            'negative_arguments': counts.get('negative', 0)}

df_counts_paragraph = df_paragraphs_arguments['provided_arguments'].apply(count_sentiments)
df_counts_paragraph = pd.DataFrame(df_counts_paragraph.tolist())  # Expand the dictionary into separate columns

df_paragraphs_arguments = pd.concat([df_paragraphs_arguments, df_counts_paragraph], axis=1)

In [1019]:
df_counts_reports = df_paragraphs_arguments.groupby('filename',as_index=False)[['positive_arguments','negative_arguments','neutral_arguments']].sum()

In [1020]:
df_reports = df_reports.merge(df_counts_reports,on="filename",how='inner',)
#df_merged = df_saved_reports.merge(df_updated_reports, on='filename', how='left', suffixes=('', '_new'))

In [1021]:
df_reports["tp_reached"] = df_reports[["tp reached after 3 months","tp reached after 6 months","tp reached after 9 months","tp reached after 12 months"]].any(axis=1)

In [1022]:
df_reports["total_arguments"] = df_reports[["positive_arguments","negative_arguments","neutral_arguments"]].sum(axis=1)

In [1023]:
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,adjusted_target_price,tp reached after 3 months,tp reached after 6 months,tp reached after 9 months,tp reached after 12 months,positive_arguments,negative_arguments,neutral_arguments,tp_reached,total_arguments
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,150.00,False,False,True,True,15,3,2,True,20
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,300.00,True,False,True,True,19,9,1,True,29
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,39.00,True,True,True,True,21,0,1,True,22
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,1950.00,False,True,True,True,7,5,2,True,14
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,50.00,False,False,False,False,15,2,1,False,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,NaN,False,False,False,False,14,4,0,False,18
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,270.00,True,True,True,True,15,3,1,True,19
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,55.00,True,True,True,True,16,10,5,True,31
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,44.00,False,False,False,True,38,12,9,True,59


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument Mapping</h2>
</div>

In [1024]:
df_arguments_sentiment.to_csv("full_list_arguments_3.5.csv")

In [1025]:
df_arguments_sentiment

,Argument,positive,neutral,negative
0,Technology Trends,527,29,54
1,Market Expansion,509,19,49
2,Partnerships and Collaborations,130,2,1
3,Innovation,367,8,19
4,Competition,91,60,573
...,...,...,...,...
380,Market Recognition,1,0,0
381,Diversity,1,0,0
382,Integration,1,0,0
383,Store Expansion,2,0,0


In [1026]:
#Has to be done later    
#Plan: Assign similar categories to the same broad category

# Create map to merge very close arguments
mapping = {
    "Growth":["Sales Growth","Earnings Growth"],
    "Price/Earnings Ratio":[],
    "Earnings per Share":["EPS"],
    "Cash Flow":["Cash Position","Cash Management","Liquidity","Working Capital"],
    "Revenue":["Revenue Growth","Sales","Operating Income","Deferred Revenue","Revenue Recognition"],
    "Return On Equity":["Return On Invested Capital","Return On Tangible Common Equity","Return On Assets","Return On Capital Employed"],
    "Margins":["EBITDA","Gross Margin","Net Income","EBIT Margin","Gross Profits","Net Interest Margin","Margins Expansion","Gross Margin Return on Inventory","EBITDA Margin"],
    "Cost Management":["Expenses","Operating Expenses","SG&A","Operating Expense","Spending","Cost Reduction"],
    "Dividend Policy":[],
    "Investments":["Capital Expenditure","Capex Allocation"],
    "Balance Sheet":["Capital Allocation","Debt Reduction","Capital Adequacy","Asset Sale","Equities","Asset Management","Loan Growth"],
    "Long-term Growth":[],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Refranchising":["Franchise Management"],
    "Sustainability":["Environmental Impact","Emissions Reduction","Energy Transition"],
    "Employees":["Intellectual Property","Talent Management","Labor Challenges","Employee Satisfaction"],
    "Research and Development":["Product Development","Drug Development","Product Launch"],
    "Marketing":["Advertising","Brand Image","Promotions"],
    "Shares Repurchase":["Buybacks", "Share Buyback", "Share rebuy"],
    "Processes":[],
    "Leadership":["Management"],
    "Innovation":["Product Innovation"],
    "Product Characteristics":[],
    "Pricing Strategy":[],
    "Production":[],
    "Technology Trends":["Digital Transformation","Digital Initiatives"],
    "Market Share":[],
    "Market Conditions":[],
    "Market Expansion":["International Expansion"],
    "Competition":["Competitive Position"],
    "Global Presence":["Global Expansion"],
    "Industry Outlook":["Market Trends"],
    "Regulations":["Regulatory Issues","Legal Issues","Privacy","Political Environment"],
    "Partnerships and Collaborations":["Partnerships","Collaboration","Partnerships and Collaboration","Collaborations"],
    "Supply Chain":[],
    "Economic Conditions":["Interest Rates","Net Interest Income","Inflation", "Global Economic Conditions"],
    "Demand":["User Growth","Market Growth"],
    "Customer":["Customer Experience","Customer","Customer Acquisition","Customer Service","Customer Retention", "Customer Engagement","User Engagement","Customer","Customer Protection"],
    "Currency":["Currency Exchange","Currency Impact","Foreign Exchange"],
    "Taxes":["Tax Rate","Tax Policy Impact","Tax Liability","Tax Impact","Taxes"],
    "Inventory Management":["Inventory"],
    "Earnings Forecast":["Earnings Guidance"],
    "Financial Performance":["Financial Strength","Earnings Quality"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_arguments_sentiment["Argument"] = df_arguments_sentiment["Argument"].replace(reverse_mapping)

df_grouped_arguments = df_arguments_sentiment.groupby("Argument").sum(numeric_only=True).reset_index()

# Display the result
df_grouped_arguments

,Argument,positive,neutral,negative
0,API Integration,1,0,0
1,Accounting Practices,0,1,1
2,Ad ARPU,1,0,0
3,Apparel Recovery,0,0,1
4,Asset-Light Models,1,0,0
...,...,...,...,...
289,Volume Growth,1,0,1
290,Volume Leverage,2,0,0
291,Workflow Integration,1,0,0
292,XPO Interest,0,1,0


# Sentiment Analysis

In [1027]:
# Get dataframe for successful recommendations
df_sentiment_analysis = df_grouped_arguments

# Display the overview DataFrame
df_sentiment_analysis.reset_index(inplace=True)
df_sentiment_analysis["total"] = df_sentiment_analysis[["positive","negative","neutral"]].sum(axis=1)
df_sentiment_analysis = df_sentiment_analysis[df_sentiment_analysis["total"] > 3 ]

In [1028]:
df_sentiment_analysis

,index,Argument,positive,neutral,negative,total
5,5,Backlog,7,0,1,8
6,6,Balance Sheet,202,53,53,308
9,9,Business Transformation,4,0,0,4
10,10,COVID Impact,3,0,6,9
14,14,Cash Flow,306,39,110,455
...,...,...,...,...,...,...
268,268,Trading,6,0,1,7
283,283,Valuation,87,210,117,414
284,284,Value Creation,4,0,0,4
285,285,Value Investing,4,0,1,5


In [1029]:
# Create Ratio fields
df_sentiment_analysis["argument_importance"] = round((df_sentiment_analysis["total"] / df_sentiment_analysis["total"].sum()),2)
df_sentiment_analysis["positive_ratio"] = round((df_sentiment_analysis["positive"] / df_sentiment_analysis["total"]),2)
df_sentiment_analysis["negative_ratio"] = round((df_sentiment_analysis["negative"] / df_sentiment_analysis["total"]),2)
df_sentiment_analysis["neutral_ratio"] = round((df_sentiment_analysis["neutral"] / df_sentiment_analysis["total"]),2)
df_sentiment_analysis["opinion_score"] = round((df_sentiment_analysis["positive"] / df_sentiment_analysis["total"]),2) - round((df_sentiment_analysis["negative"] / df_sentiment_analysis["total"]),2)

#df_sentiment_analysis["positive"] = df_sentiment_analysis["positive"].apply(lambda x: x.int())
df_sentiment_analysis = df_sentiment_analysis.sort_values(by=["total"],ascending=False)
df_sentiment_analysis["position"] = range(1,len(df_sentiment_analysis)+1)
df_sentiment_analysis.head(30)

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_1408/236307676.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment_analysis["argument_importance"] = round((df_sentiment_analysis["total"] / df_sentiment_analysis["total"].sum()),2)
/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_1408/236307676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment_analysis["positive_ratio"] = round((df_sentiment_analysis["positive"] / df_sentiment_analysis["total"]),2)
/var/folders

,index,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,position
123,123,Growth,2621,100,404,3125,0.13,0.84,0.13,0.03,0.71,1
221,221,Revenue,2102,272,593,2967,0.12,0.71,0.20,0.09,0.51,2
157,157,Margins,788,90,538,1416,0.06,0.56,0.38,0.06,0.18,3
165,165,Market Share,938,47,189,1174,0.05,0.80,0.16,0.04,0.64,4
159,159,Market Conditions,394,172,550,1116,0.05,0.35,0.49,0.15,-0.14,5
138,138,Investments,615,140,169,924,0.04,0.67,0.18,0.15,0.49,6
32,32,Cost Management,447,77,354,878,0.04,0.51,0.40,0.09,0.11,7
20,20,Competition,141,60,591,792,0.03,0.18,0.75,0.08,-0.57,8
80,80,Earnings per Share,452,127,123,702,0.03,0.64,0.18,0.18,0.46,9
267,267,Technology Trends,554,29,54,637,0.03,0.87,0.08,0.05,0.79,10


In [1030]:
df_success_evaluation_reports = pd.merge(df_paragraphs_arguments,df_reports[["filename","tp_reached","rating","industry"]],on="filename",how='left')

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Successful Reports</h2>
</div>

In [1031]:
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

In [1032]:
#Should I reassign the arguments here??

# Get dataframe for successful recommendations
df_success_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==True]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_success = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_success_reports["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts_success[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
df_sentiment_success = pd.DataFrame(sentiment_counts_success).T
df_sentiment_success.columns = ["positive", "neutral", "negative"]
df_sentiment_success.index.name = "Argument"

# Display the overview DataFrame
df_sentiment_success.reset_index(inplace=True)
df_sentiment_success["total"] = df_sentiment_success[["positive","negative","neutral"]].sum(axis=1)
df_sentiment_success

,Argument,positive,neutral,negative,total
0,Market Share,475,28,92,595
1,Revenue,867,107,220,1194
2,Growth,1102,32,104,1238
3,Margins,334,39,240,613
4,Competition,46,31,273,350
...,...,...,...,...,...
303,Tech Spending,1,1,0,2
304,Interest Expense,0,0,1,1
305,Volume Growth,0,0,1,1
306,Consumer Trends,0,0,1,1


In [1033]:
## Mapping before ratios are getting created
mapping = {
    "Growth":["Sales Growth","Earnings Growth"],
    "Price/Earnings Ratio":[],
    "Earnings per Share":["EPS"],
    "Cash Flow":["Cash Position","Cash Management","Liquidity","Working Capital"],
    "Revenue":["Revenue Growth","Sales","Operating Income","Deferred Revenue","Revenue Recognition"],
    "Return On Equity":["Return On Invested Capital","Return On Tangible Common Equity","Return On Assets","Return On Capital Employed"],
    "Margins":["EBITDA","Gross Margin","Net Income","EBIT Margin","Gross Profits","Net Interest Margin","Margins Expansion","Gross Margin Return on Inventory","EBITDA Margin"],
    "Cost Management":["Expenses","Operating Expenses","SG&A","Operating Expense","Spending","Cost Reduction"],
    "Dividend Policy":[],
    "Investments":["Capital Expenditure","Capex Allocation"],
    "Balance Sheet":["Capital Allocation","Debt Reduction","Capital Adequacy","Asset Sale","Equities","Asset Management","Loan Growth"],
    "Long-term Growth":[],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Refranchising":["Franchise Management"],
    "Sustainability":["Environmental Impact","Emissions Reduction","Energy Transition"],
    "Employees":["Intellectual Property","Talent Management","Labor Challenges","Employee Satisfaction"],
    "Research and Development":["Product Development","Drug Development","Product Launch"],
    "Marketing":["Advertising","Brand Image","Promotions"],
    "Shares Repurchase":["Buybacks", "Share Buyback", "Share rebuy"],
    "Processes":[],
    "Leadership":["Management"],
    "Innovation":["Product Innovation"],
    "Product Characteristics":[],
    "Pricing Strategy":[],
    "Production":[],
    "Technology Trends":["Digital Transformation","Digital Initiatives"],
    "Market Share":[],
    "Market Conditions":[],
    "Market Expansion":["International Expansion"],
    "Competition":["Competitive Position"],
    "Global Presence":["Global Expansion"],
    "Industry Outlook":["Market Trends"],
    "Regulations":["Regulatory Issues","Legal Issues","Privacy","Political Environment"],
    "Partnerships and Collaborations":["Partnerships","Collaboration","Partnerships and Collaboration","Collaborations"],
    "Supply Chain":[],
    "Economic Conditions":["Interest Rates","Net Interest Income","Inflation", "Global Economic Conditions"],
    "Demand":["User Growth","Market Growth"],
    "Customer":["Customer Experience","Customer","Customer Acquisition","Customer Service","Customer Retention", "Customer Engagement","User Engagement","Customer","Customer Protection"],
    "Currency":["Currency Exchange","Currency Impact","Foreign Exchange"],
    "Taxes":["Tax Rate","Tax Policy Impact","Tax Liability","Tax Impact","Taxes"],
    "Inventory Management":["Inventory"],
    "Earnings Forecast":["Earnings Guidance"],
    "Financial Performance":["Financial Strength","Earnings Quality"]
}
reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_sentiment_success["Argument"] = df_sentiment_success["Argument"].replace(reverse_mapping)

# Group By the broad category to assign the numbers
df_sentiment_success = df_sentiment_success.groupby("Argument").sum(numeric_only=True).reset_index()
df_sentiment_success

,Argument,positive,neutral,negative,total
0,API Integration,1,0,0,1
1,Accounting Practices,0,1,1,2
2,Ad ARPU,1,0,0,1
3,Apparel Recovery,0,0,1,1
4,Asset-Light Models,1,0,0,1
...,...,...,...,...,...
216,Visibility,0,1,3,4
217,Volume,0,0,1,1
218,Volume Growth,0,0,1,1
219,Workflow Integration,1,0,0,1


In [1034]:
# Create Ratio fields
df_sentiment_success["argument_importance"] = round((df_sentiment_success["total"] / df_sentiment_success["total"].sum()),2)
df_sentiment_success["positive_ratio"] = round((df_sentiment_success["positive"] / df_sentiment_success["total"]),2)
df_sentiment_success["negative_ratio"] = round((df_sentiment_success["negative"] / df_sentiment_success["total"]),2)
df_sentiment_success["neutral_ratio"] = round((df_sentiment_success["neutral"] / df_sentiment_success["total"]),2)
df_sentiment_success["opinion_score"] = round((df_sentiment_success["positive"] / df_sentiment_success["total"]),2) - round((df_sentiment_success["negative"] / df_sentiment_success["total"]),2)

#df_sentiment_success["positive"] = df_sentiment_success["positive"].apply(lambda x: x.int())
df_sentiment_success = df_sentiment_success.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_success["position"] = range(1,len(df_sentiment_success)+1)
df_sentiment_success.head(30)

,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,position
94,Growth,1284,43,187,1514,0.13,0.85,0.12,0.03,0.73,1
170,Revenue,960,115,239,1314,0.11,0.73,0.18,0.09,0.55,2
125,Market Share,475,28,92,595,0.05,0.80,0.15,0.05,0.65,3
118,Margins,362,45,251,658,0.06,0.55,0.38,0.07,0.17,4
204,Technology Trends,303,16,26,345,0.03,0.88,0.08,0.05,0.80,5
106,Investments,259,65,70,394,0.03,0.66,0.18,0.16,0.48,6
122,Market Expansion,259,9,26,294,0.03,0.88,0.09,0.03,0.79,7
21,Cost Management,208,31,158,397,0.03,0.52,0.40,0.08,0.12,8
101,Innovation,205,7,17,229,0.02,0.90,0.07,0.03,0.83,9
56,Earnings per Share,194,53,44,291,0.03,0.67,0.15,0.18,0.52,10


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Not Successful Reports</h2>
</div>

In [1035]:
df_unsuccess_reports = df_success_evaluation_reports[df_success_evaluation_reports["tp_reached"]==False]

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts_unsuccess = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_unsuccess_reports["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts_unsuccess[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
df_sentiment_unsuccess = pd.DataFrame(sentiment_counts_unsuccess).T
df_sentiment_unsuccess.columns = ["positive", "neutral", "negative"]
df_sentiment_unsuccess.index.name = "Argument"

# Display the overview DataFrame
df_sentiment_unsuccess.reset_index(inplace=True)
df_sentiment_unsuccess["total"] = df_sentiment_unsuccess[["positive","negative","neutral"]].sum(axis=1)
df_sentiment_unsuccess

,Argument,positive,neutral,negative,total
0,Technology Trends,240,13,28,281
1,Market Expansion,257,10,27,294
2,Partnerships and Collaborations,69,2,1,72
3,Innovation,180,3,3,186
4,Competition,45,29,300,374
...,...,...,...,...,...
319,Integration,1,0,0,1
320,Store Expansion,2,0,0,2
321,Emissions Reduction,1,0,0,1
322,Compliance,0,0,1,1


In [1036]:
## Mapping before ratios are getting created
mapping = {
    "Growth":["Sales Growth","Earnings Growth"],
    "Price/Earnings Ratio":[],
    "Earnings per Share":["EPS"],
    "Cash Flow":["Cash Position","Cash Management","Liquidity","Working Capital"],
    "Revenue":["Revenue Growth","Sales","Operating Income","Deferred Revenue","Revenue Recognition"],
    "Return On Equity":["Return On Invested Capital","Return On Tangible Common Equity","Return On Assets","Return On Capital Employed"],
    "Margins":["EBITDA","Gross Margin","Net Income","EBIT Margin","Gross Profits","Net Interest Margin","Margins Expansion","Gross Margin Return on Inventory","EBITDA Margin"],
    "Cost Management":["Expenses","Operating Expenses","SG&A","Operating Expense","Spending","Cost Reduction"],
    "Dividend Policy":[],
    "Investments":["Capital Expenditure","Capex Allocation"],
    "Balance Sheet":["Capital Allocation","Debt Reduction","Capital Adequacy","Asset Sale","Equities","Asset Management","Loan Growth"],
    "Long-term Growth":[],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Refranchising":["Franchise Management"],
    "Sustainability":["Environmental Impact","Emissions Reduction","Energy Transition"],
    "Employees":["Intellectual Property","Talent Management","Labor Challenges","Employee Satisfaction"],
    "Research and Development":["Product Development","Drug Development","Product Launch"],
    "Marketing":["Advertising","Brand Image","Promotions"],
    "Shares Repurchase":["Buybacks", "Share Buyback", "Share rebuy"],
    "Processes":[],
    "Leadership":["Management"],
    "Innovation":["Product Innovation"],
    "Product Characteristics":[],
    "Pricing Strategy":[],
    "Production":[],
    "Technology Trends":["Digital Transformation","Digital Initiatives"],
    "Market Share":[],
    "Market Conditions":[],
    "Market Expansion":["International Expansion"],
    "Competition":["Competitive Position"],
    "Global Presence":["Global Expansion"],
    "Industry Outlook":["Market Trends"],
    "Regulations":["Regulatory Issues","Legal Issues","Privacy","Political Environment"],
    "Partnerships and Collaborations":["Partnerships","Collaboration","Partnerships and Collaboration","Collaborations"],
    "Supply Chain":[],
    "Economic Conditions":["Interest Rates","Net Interest Income","Inflation", "Global Economic Conditions"],
    "Demand":["User Growth","Market Growth"],
    "Customer":["Customer Experience","Customer","Customer Acquisition","Customer Service","Customer Retention", "Customer Engagement","User Engagement","Customer","Customer Protection"],
    "Currency":["Currency Exchange","Currency Impact","Foreign Exchange"],
    "Taxes":["Tax Rate","Tax Policy Impact","Tax Liability","Tax Impact","Taxes"],
    "Inventory Management":["Inventory"],
    "Earnings Forecast":["Earnings Guidance"],
    "Financial Performance":["Financial Strength","Earnings Quality"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_sentiment_unsuccess["Argument"] = df_sentiment_unsuccess["Argument"].replace(reverse_mapping)

# Group By the broad category to assign the numbers
df_sentiment_unsuccess = df_sentiment_unsuccess.groupby("Argument").sum(numeric_only=True).reset_index()
df_sentiment_unsuccess

,Argument,positive,neutral,negative,total
0,Backlog,7,0,1,8
1,Balance Sheet,109,22,23,154
2,Brokers Uncertainty,0,1,0,1
3,Business Transformation,3,0,0,3
4,COVID Impact,3,0,6,9
...,...,...,...,...,...
230,Value Unlocking,1,0,0,1
231,Visibility,1,0,1,2
232,Volume Growth,1,0,0,1
233,Volume Leverage,2,0,0,2


In [1037]:
# Create Ratio fields
df_sentiment_unsuccess["argument_importance"] = round((df_sentiment_unsuccess["total"] / df_sentiment_unsuccess["total"].sum()),2)
df_sentiment_unsuccess["positive_ratio"] = round((df_sentiment_unsuccess["positive"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess["negative_ratio"] = round((df_sentiment_unsuccess["negative"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess["neutral_ratio"] = round((df_sentiment_unsuccess["neutral"] / df_sentiment_unsuccess["total"]),2)
df_sentiment_unsuccess["opinion_score"] = round((df_sentiment_unsuccess["positive"] / df_sentiment_unsuccess["total"]),2) - round((df_sentiment_unsuccess["negative"] / df_sentiment_unsuccess["total"]),2)

df_sentiment_unsuccess = df_sentiment_unsuccess.sort_values(by=["positive","negative","neutral"],ascending=False)
df_sentiment_unsuccess["position"] = range(1,len(df_sentiment_unsuccess)+1)

df_sentiment_unsuccess.head(30)

,Argument,positive,neutral,negative,total,argument_importance,positive_ratio,negative_ratio,neutral_ratio,opinion_score,position
91,Growth,1337,57,217,1611,0.12,0.83,0.13,0.04,0.70,1
174,Revenue,1142,157,354,1653,0.12,0.69,0.21,0.09,0.48,2
128,Market Share,463,19,97,579,0.04,0.80,0.17,0.03,0.63,3
120,Margins,426,45,287,758,0.06,0.56,0.38,0.06,0.18,4
104,Investments,356,75,99,530,0.04,0.67,0.19,0.14,0.48,5
124,Market Expansion,267,11,29,307,0.02,0.87,0.09,0.04,0.78,6
60,Earnings per Share,258,74,79,411,0.03,0.63,0.19,0.18,0.44,7
215,Technology Trends,251,13,28,292,0.02,0.86,0.10,0.04,0.76,8
24,Cost Management,239,46,196,481,0.04,0.50,0.41,0.10,0.09,9
97,Innovation,209,4,9,222,0.02,0.94,0.04,0.02,0.90,10


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Comparison of Success</h2>
</div>

https://stackoverflow.com/questions/52783391/how-to-merge-dictionaries-of-a-pandas-dataframe-when-grouping-by-rows

In [1038]:
# Merge the two DataFrames on 'Argument' to align rows
merged_df = pd.merge(df_sentiment_success, df_sentiment_unsuccess, on='Argument', suffixes=('_df1', '_df2'))
merged_df["average_total"] = (merged_df["total_df1"] + merged_df["total_df2"])/2

In [1039]:
merged_df.head(30)

,Argument,positive_df1,neutral_df1,negative_df1,total_df1,argument_importance_df1,positive_ratio_df1,negative_ratio_df1,neutral_ratio_df1,opinion_score_df1,...,neutral_df2,negative_df2,total_df2,argument_importance_df2,positive_ratio_df2,negative_ratio_df2,neutral_ratio_df2,opinion_score_df2,position_df2,average_total
0,Growth,1284,43,187,1514,0.13,0.85,0.12,0.03,0.73,...,57,217,1611,0.12,0.83,0.13,0.04,0.70,1,1562.5
1,Revenue,960,115,239,1314,0.11,0.73,0.18,0.09,0.55,...,157,354,1653,0.12,0.69,0.21,0.09,0.48,2,1483.5
2,Market Share,475,28,92,595,0.05,0.80,0.15,0.05,0.65,...,19,97,579,0.04,0.80,0.17,0.03,0.63,3,587.0
3,Margins,362,45,251,658,0.06,0.55,0.38,0.07,0.17,...,45,287,758,0.06,0.56,0.38,0.06,0.18,4,708.0
4,Technology Trends,303,16,26,345,0.03,0.88,0.08,0.05,0.80,...,13,28,292,0.02,0.86,0.10,0.04,0.76,8,318.5
5,Investments,259,65,70,394,0.03,0.66,0.18,0.16,0.48,...,75,99,530,0.04,0.67,0.19,0.14,0.48,5,462.0
6,Market Expansion,259,9,26,294,0.03,0.88,0.09,0.03,0.79,...,11,29,307,0.02,0.87,0.09,0.04,0.78,6,300.5
7,Cost Management,208,31,158,397,0.03,0.52,0.40,0.08,0.12,...,46,196,481,0.04,0.50,0.41,0.10,0.09,9,439.0
8,Innovation,205,7,17,229,0.02,0.90,0.07,0.03,0.83,...,4,9,222,0.02,0.94,0.04,0.02,0.90,10,225.5
9,Earnings per Share,194,53,44,291,0.03,0.67,0.15,0.18,0.52,...,74,79,411,0.03,0.63,0.19,0.18,0.44,7,351.0


In [1040]:
# Calculate differences for all numeric columns (excluding 'Argument')
difference_df = merged_df[['Argument',"average_total"]].copy()  # Start with the 'Argument' column
columns_to_compare = [col for col in df_sentiment_success.columns if col != 'Argument' and col != 'average_total']  # Columns to compute differences for

for col in columns_to_compare:
    difference_df[col + '_difference'] = merged_df[col + '_df1'] - merged_df[col + '_df2']



In [1041]:
difference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Argument                        162 non-null    object 
 1   average_total                   162 non-null    float64
 2   positive_difference             162 non-null    int64  
 3   neutral_difference              162 non-null    int64  
 4   negative_difference             162 non-null    int64  
 5   total_difference                162 non-null    int64  
 6   argument_importance_difference  162 non-null    float64
 7   positive_ratio_difference       162 non-null    float64
 8   negative_ratio_difference       162 non-null    float64
 9   neutral_ratio_difference        162 non-null    float64
 10  opinion_score_difference        162 non-null    float64
 11  position_difference             162 non-null    int64  
dtypes: float64(6), int64(5), object(1)
m

In [1042]:
difference_df.to_csv("difference_testing.csv")

In [1043]:
difference_df

,Argument,average_total,positive_difference,neutral_difference,negative_difference,total_difference,argument_importance_difference,positive_ratio_difference,negative_ratio_difference,neutral_ratio_difference,opinion_score_difference,position_difference
0,Growth,1562.5,-53,-14,-30,-97,0.01,0.02,-0.01,-0.01,0.03,0
1,Revenue,1483.5,-182,-42,-115,-339,-0.01,0.04,-0.03,0.00,0.07,0
2,Market Share,587.0,12,9,-5,16,0.01,0.00,-0.02,0.02,0.02,0
3,Margins,708.0,-64,0,-36,-100,0.00,-0.01,0.00,0.01,-0.01,0
4,Technology Trends,318.5,52,3,-2,53,0.01,0.02,-0.02,0.01,0.04,-3
...,...,...,...,...,...,...,...,...,...,...,...,...
157,Commercial Lending,1.5,-1,1,-1,-1,0.00,-0.50,-0.50,1.00,0.00,79
158,Diversification,1.5,-2,1,0,-1,0.00,-1.00,0.00,1.00,-1.00,99
159,Expense,1.5,0,0,-1,-1,0.00,0.00,-0.50,0.50,0.50,23
160,Fees,1.5,-1,1,-1,-1,0.00,-0.50,-0.50,1.00,0.00,82


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Statistical Significance</h2>
</div>
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

In [1044]:
import numpy as np

def cohens_d(group1, group2):
    # Means of the groups
    mean1, mean2 = np.mean(group1), np.mean(group2)
    
    # Standard deviations of the groups
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # Sizes of the groups
    n1, n2 = len(group1), len(group2)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    
    # Compute Cohen's d
    return (mean1 - mean2) / pooled_std

In [1045]:
import numpy as np

# Extract total success and unsuccess counts
success_counts = merged_df["total_df1"].tolist()
unsuccess_counts = merged_df["total_df2"].tolist()

cohens_d_values = []

for success, unsuccess in zip(success_counts, unsuccess_counts):
    # Means are the counts themselves
    mean_success = success
    mean_unsuccess = unsuccess
    
    # Approximate standard deviations (for count data, use sqrt(count))
    std_success = np.sqrt(success)  # Estimated standard deviation for success
    std_unsuccess = np.sqrt(unsuccess)  # Estimated standard deviation for unsuccess
    
    # Compute pooled standard deviation
    pooled_std = np.sqrt(((std_success**2 + std_unsuccess**2) / 2))
    
    # Compute Cohen's d
    d = (mean_success - mean_unsuccess) / pooled_std
    cohens_d_values.append(d)

# Add Cohen's d values to the DataFrame
merged_df["cohen_d"] = cohens_d_values

# Display results
print(merged_df[["Argument", "cohen_d"]])

               Argument   cohen_d
0                Growth -2.453927
1               Revenue -8.801484
2          Market Share  0.660391
3               Margins -3.758230
4     Technology Trends  2.969759
..                  ...       ...
157  Commercial Lending -0.816497
158     Diversification -0.816497
159             Expense -0.816497
160                Fees -0.816497
161          Securities  0.000000

[162 rows x 2 columns]


In [1046]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest
import pingouin as pg


# Step 1: Chi-square test for overall usage
merged_df["relative_usage_df1"] = merged_df["total_df1"] / merged_df["total_df1"].sum()
merged_df["relative_usage_df2"] = merged_df["total_df2"] / merged_df["total_df2"].sum()

# Create lists to store test results
statistical_significance_values = []

# Loop through each argument
for _, row in merged_df.iterrows():
    argument = row["Argument"]

    successful_count = row["total_df1"]
    not_successful_count = row["total_df2"]
    other_successful_count = sum(merged_df["total_df1"]) - successful_count
    other_not_successful_count = sum(merged_df["total_df2"]) - not_successful_count

    contingency_table = [[successful_count, other_successful_count],
                     [not_successful_count, other_not_successful_count]]

    chi2_stat, p_total, _, _ = stats.chi2_contingency(contingency_table)

    print(contingency_table)

    # Step 2: Proportion z-test for relative usage
    count = [row["relative_usage_df1"] * sum(merged_df["total_df1"]),
             row["relative_usage_df2"] * sum(merged_df["total_df2"])]
    nobs = [sum(merged_df["total_df1"]), sum(merged_df["total_df2"])]
    z_stat, p_relative = proportions_ztest(count, nobs)

    # Compute Cohen's d for each argument

    # Store results
    statistical_significance_values.append({
        "Argument": argument,
        "p_total": p_total,
        "p_relative": p_relative,
        "z_stat":z_stat,
        "success_count":row["total_df1"],
        "unsuccess_count":row["total_df2"],
        "cohen_d":row["cohen_d"]
    })

# Create a DataFrame for results
df_statistical_significance = pd.DataFrame(statistical_significance_values)

# Display the results
df_statistical_significance.head(30)

[[1514, 9972], [1611, 11567]]
[[1314, 10172], [1653, 11525]]
[[595, 10891], [579, 12599]]
[[658, 10828], [758, 12420]]
[[345, 11141], [292, 12886]]
[[394, 11092], [530, 12648]]
[[294, 11192], [307, 12871]]
[[397, 11089], [481, 12697]]
[[229, 11257], [222, 12956]]
[[291, 11195], [411, 12767]]
[[495, 10991], [621, 12557]]
[[275, 11211], [322, 12856]]
[[158, 11328], [190, 12988]]
[[168, 11318], [184, 12994]]
[[211, 11275], [289, 12889]]
[[186, 11300], [269, 12909]]
[[133, 11353], [153, 13025]]
[[154, 11332], [154, 13024]]
[[143, 11343], [150, 13028]]
[[101, 11385], [87, 13091]]
[[107, 11379], [66, 13112]]
[[83, 11403], [92, 13086]]
[[384, 11102], [408, 12770]]
[[102, 11384], [135, 13043]]
[[82, 11404], [116, 13062]]
[[167, 11319], [172, 13006]]
[[105, 11381], [140, 13038]]
[[57, 11429], [33, 13145]]
[[66, 11420], [55, 13123]]
[[197, 11289], [217, 12961]]
[[97, 11389], [115, 13063]]
[[163, 11323], [170, 13008]]
[[51, 11435], [51, 13127]]
[[48, 11438], [51, 13127]]
[[37, 11449], [39, 13139]

,Argument,p_total,p_relative,z_stat,success_count,unsuccess_count,cohen_d
0,Growth,0.025545,0.024306,2.252253,1514,1611,-2.453927
1,Revenue,0.008338,0.007868,-2.657673,1314,1653,-8.801484
2,Market Share,0.004184,0.003805,2.893892,595,579,0.660391
3,Margins,0.959310,0.937465,-0.078457,658,758,-3.758230
4,Technology Trends,0.000118,0.000100,3.890996,345,292,2.969759
5,Investments,0.016087,0.014665,-2.440544,394,530,-6.327293
6,Market Expansion,0.259672,0.242580,1.168562,294,307,-0.749931
7,Cost Management,0.432885,0.412952,-0.818711,397,481,-4.009101
8,Innovation,0.078461,0.070712,1.807321,229,222,0.466149
9,Earnings per Share,0.006547,0.005826,-2.757406,291,411,-6.405126


## Create new columns
1. Add TP_Reached_as one column
2. Add total_as_arguments one column

# 2. Replace similar arguments with broader category

# 3. Calculate number of positive / negative / neutral Arguments

# Last Step: Merge Dictionaries

# Report Analysis

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Report Analysis</h2>
</div>

In [1047]:
#1. Step -> create additional columns that contain the arguments/sentiments as a list, this way the argument can be multiple times per document (not possibel in dictionary)
df_paragraphs_arguments['argument_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.keys()))
df_paragraphs_arguments['sentiment_list'] = df_paragraphs_arguments['provided_arguments'].apply(lambda x: list(x.values()))

In [1048]:
## Mapping before ratios are getting created
mapping = {
    "Shares Repurchase": ["Buybacks", "Share Buyback", "Share rebuy"],
    "Partnerships and Collaboration":["Partnerships","Collaboration"],
    "Pricing Strategy":["Pricing"],
    "Mergers and Acquisition":["Acquisition","Acquisitions"],
    "Revenue":["Ad Revenue"],
    "Regulations":["Regulatory Approval"],
    "Cash Flow":["Free Cash Flow"],
    "Marketing":["Brand Strength"],
    "Competition":["Competitive Positioning"]
}

reverse_mapping = {v: k for k, values in mapping.items() for v in values}

# Reassign argument
df_paragraphs_arguments["argument_list"] = df_paragraphs_arguments["argument_list"].replace(reverse_mapping)

In [1049]:
df_paragraphs_arguments

,filename,document_id,paragraph_id,provided_arguments,positive_arguments,neutral_arguments,negative_arguments,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"{'Technology Trends': 'positive', 'Market Expa...",1,0,1,"[Technology Trends, Market Expansion]","[positive, negative]"
1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,"{'Technology Trends': 'positive', 'Partnership...",2,0,0,"[Technology Trends, Partnerships and Collabora...","[positive, positive]"
2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,"{'Innovation': 'positive', 'Technology Trends'...",3,0,0,"[Innovation, Technology Trends, Competition]","[positive, positive, positive]"
3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,"{'Technology Trends': 'positive', 'Market Shar...",2,0,1,"[Technology Trends, Market Share, Competition]","[positive, positive, negative]"
4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,"{'Growth': 'negative', 'Technology Trends': 'n...",0,0,3,"[Growth, Technology Trends, Competition]","[negative, negative, negative]"
...,...,...,...,...,...,...,...,...,...
12567,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,"{'Market Share': 'positive', 'Revenue': 'negat...",1,0,1,"[Market Share, Revenue]","[positive, negative]"
12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"{'Market Share': 'positive', 'Growth': 'positi...",2,0,0,"[Market Share, Growth]","[positive, positive]"
12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,{'Market Share': 'positive'},1,0,0,[Market Share],[positive]
12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,{'Growth': 'positive'},1,0,0,[Growth],[positive]


In [1050]:
#2. Step -> Merge argument_list and sentiment_list on document level 
# merge dictionaries
def merge_lists(x):
    return [item for sublist in x.dropna() for item in sublist]

df_reports_arguments = df_paragraphs_arguments.groupby(['filename', 'document_id'], as_index=False).agg({
    'argument_list': merge_lists,
    'sentiment_list': merge_lists
})

# Display the result
df_reports_arguments

,filename,document_id,argument_list,sentiment_list
0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,"[Technology Trends, Market Expansion, Technolo...","[positive, negative, positive, positive, posit..."
1,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,909,"[Market Share, Revenue, Growth, Revenue, Margi...","[positive, positive, positive, positive, negat..."
2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,717,"[Competition, Margins, Competition, Revenue, M...","[negative, negative, negative, negative, negat..."
3,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,917,"[Margins, Long-term Growth, Investments, Margi...","[negative, negative, negative, negative, posit..."
4,20140124_Hilliard_Lyons_PG_Report_received_in_...,574,"[Revenue, Market Share, Margins, Earnings per ...","[positive, positive, negative, positive, posit..."
...,...,...,...,...
911,20221227_JP_Morgan_WFC_Wells_Fargo-_A_New_Surp...,793,"[Regulatory Issues, Growth, Regulatory Issues,...","[negative, negative, negative, negative, negat..."
912,20221228_JP_Morgan_UAA_Under_Armour-_Inc.-_Mgm...,908,"[Leadership, Growth, Market Share, Growth, Mar...","[positive, positive, negative, positive, posit..."
913,20221228_Wells_Fargo_FDX_FDX-_Good_News-_Bad_N...,642,"[Margins, Cost Management, Earnings Forecast, ...","[positive, neutral, negative, positive, neutra..."
914,20221229_JP_Morgan_MUR_Murphy-_The_Road_Ahead_...,22,"[Stock Performance, Cash Flow, Dividend Policy...","[positive, positive, positive, positive, negat..."


In [1051]:
from collections import Counter

# Function to get top 3 most frequent arguments
def get_top_arguments(arg_list, n):
    counter = Counter(arg_list)  # Count occurrences of each argument
    most_common = [item[0] for item in counter.most_common(n)]  # Extract top `n` arguments
    return most_common

# Apply the function to argument_list and create new columns
df_reports_arguments['1st_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 1)[0] if len(get_top_arguments(x, 1)) > 0 else None)
df_reports_arguments['2nd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 2)[1] if len(get_top_arguments(x, 2)) > 1 else None)
df_reports_arguments['3rd_most_used_argument'] = df_reports_arguments['argument_list'].apply(lambda x: get_top_arguments(x, 3)[2] if len(get_top_arguments(x, 3)) > 2 else None)

In [1052]:
df_reports_arguments.value_counts("1st_most_used_argument")

1st_most_used_argument
Revenue                  242
Growth                   189
Margins                   69
Market Conditions         57
Sales Growth              38
                        ... 
Management                 1
Net Interest Income        1
Operating Performance      1
Price/Mix Strategy         1
Long-term Growth           1
Name: count, Length: 66, dtype: int64

In [1053]:
df_reports_arguments.value_counts("2nd_most_used_argument")

2nd_most_used_argument
Revenue                             142
Growth                              142
Margins                              66
Market Conditions                    63
Market Share                         48
                                   ... 
Efficacy                              1
Return On Equity                      1
Return On Tangible Common Equity      1
Economic Conditions                   1
Volume Growth                         1
Name: count, Length: 90, dtype: int64

In [1054]:
df_reports_arguments.value_counts("3rd_most_used_argument")

3rd_most_used_argument
Revenue               89
Growth                87
Market Share          63
Margins               60
Earnings per Share    53
                      ..
Legal Issues           1
Loan Growth            1
Operating Expense      1
Operating Income       1
Value Creation         1
Name: count, Length: 110, dtype: int64

In [1055]:
df_final_reports = pd.merge(df_reports,df_reports_arguments,on='filename',how='inner')
df_final_reports['year'] = pd.to_datetime(df_final_reports['date']).dt.year
df_final_reports["opinion_score"] = round((df_final_reports["positive_arguments"] / df_final_reports["total_arguments"]),2) - round((df_final_reports["negative_arguments"] / df_final_reports["total_arguments"]),2)

In [1056]:
pd.reset_option('display.max_colwidth', None)  # Don't truncate column content
df_final_reports

,document_id_x,filename,date,provider,ticker,company_name,industry,target_price,rating,start price,...,tp_reached,total_arguments,document_id_y,argument_list,sentiment_list,1st_most_used_argument,2nd_most_used_argument,3rd_most_used_argument,year,opinion_score
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,150.0,buy,128.88,...,True,20,1,"[Revenue, Earnings per Share, Operating Income...","[positive, positive, positive, positive, posit...",Revenue,Growth,Earnings per Share,2016,0.60
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,300.0,overweight,253.67,...,True,29,2,"[Revenue, Growth, Cost Management, Market Cond...","[positive, neutral, positive, negative, positi...",Revenue,Growth,Market Share,2020,0.35
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,39.0,overweight,31.18,...,True,22,3,"[Growth, Innovation, Market Expansion, Growth,...","[positive, positive, positive, positive, posit...",Growth,Innovation,Market Expansion,2020,0.95
3,4,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,1950.0,overweight,1746.79,...,True,14,4,"[Revenue, EBITDA, Market Conditions, Market Ex...","[negative, negative, positive, positive, posit...",Market Conditions,Revenue,EBITDA,2020,0.14
4,5,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,50.0,buy,23.38,...,False,18,5,"[Growth, Investments, Financial Performance, F...","[positive, positive, positive, positive, neutr...",Growth,Financial Performance,Investments,2021,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,913,20150522_Gilford_Securities_Inc_FL_Report_rece...,2015-05-22,Gilford Securities Inc,FL,Foot Locker,Clothing,NaN,buy,63.46,...,False,18,913,"[Growth, Earnings per Share, Dividend Policy, ...","[positive, positive, positive, positive, posit...",Growth,Earnings per Share,Dividend Policy,2015,0.56
912,914,20200526_Wells_Fargo_HD_HD-_Q2_Rollover_Falls_...,2020-05-19,Wells Fargo,HD,Home Depot Inc.,Consumer Discretionary,270.0,overweight,238.10,...,True,19,914,"[Market Share, Revenue, Sustainability, Financ...","[positive, positive, positive, positive, neutr...",Market Share,Market Conditions,Growth,2020,0.63
913,915,20220125_Deutsche_Bank_WFC_Wells_Fargo-_More_C...,2022-01-17,Deutsche Bank,WFC,Wells Fargo & Co.,Financials,55.0,buy,58.06,...,True,31,915,"[Earnings Forecast, Interest Rates, Cost Manag...","[positive, positive, neutral, positive, positi...",Cost Management,Loan Loss Reserves,Revenue,2022,0.20
914,916,20200212_Barclays_GM_General_Motors-_Waiting_p...,2020-02-06,Barclays,GM,General Motors Co.,Automobile,44.0,overweight,34.33,...,True,59,916,"[Valuation, Revenue, Growth, Market Cap, Earni...","[positive, positive, positive, positive, posit...",Growth,Revenue,Valuation,2020,0.44


# Opinion Score per Year & Industry

In [1057]:
df_opn_year = df_final_reports.groupby("year").agg(
    opinion_score =('opinion_score','mean')
)
df_opn_year

,opinion_score
year,
2014,0.302727
2015,0.387037
2016,0.394762
2017,0.376379
2018,0.216102
2019,0.374035
2020,0.291141
2021,0.350055
2022,0.284529


In [1058]:
df_opn_rating = df_final_reports.groupby("rating").agg(
    opinion_score =('opinion_score','mean')
)
df_opn_rating

,opinion_score
rating,
buy,0.465812
equal weight,0.145890
hold,0.141189
neutral,0.124815
none,0.096471
outperform,0.556667
overweight,0.437013
sell,-0.222857
underperform,0.426364


In [1059]:
df_opn_industry = df_final_reports.groupby("industry").agg(
    opinion_score =('opinion_score','mean')
)
df_opn_industry

,opinion_score
industry,
Automobile,0.301304
Beauty,0.297000
Clothing,0.294595
Communication Services,0.163939
Consumer Discretionary,0.425068
Consumer Staples,0.339388
Energy,0.255870
Financials,0.363438
Food,0.498939


In [1060]:
df_opn_industry_year = df_final_reports.groupby(["industry","year"]).agg(
    opinion_score =('opinion_score','mean')
)
df_opn_industry_year.head(50)

opinion_score
industry               year               
Automobile             2014       0.030000
                       2015       0.700000
                       2018       0.005000
                       2019       0.235000
                       2020       0.551429
                       2021       0.350667
                       2022       0.216875
Beauty                 2015       0.270000
                       2017      -0.140000
                       2019       0.240000
                       2020       0.091250
                       2021       0.522000
                       2022       0.293750
Clothing               2014       0.618333
                       2015       0.566000
                       2016       0.500000
                       2017       0.085000
                       2018       0.165000
                       2019       0.255000
                       2020       0.147143
                       2021       0.446250
                       2022       0.392000
Communication Services 2014       0.118571
                       2015       0.092000
                       2016       0.620000
                       2017       0.011667
                       2018      -0.082857
                       2019       0.390000
                       2020       0.055833
                       2021       0.113636
                       2022       0.421538
Consumer Discretionary 2014       0.078333
                       2015       0.435000
                       2016       0.358571
                       2017       0.548889
                       2018       0.484286
                       2019       0.452500
                       2020       0.592143
                       2021       0.538750
                       2022       0.255000
Consumer Staples       2014       0.330000
                       2015       0.485000
                       2018      -0.340000
                       2019       0.115000
                       2020       0.340000
                       2021       0.300909
                       2022       0.406000
Energy                 2014       0.070000
                       2015       0.325000
                       2017      -0.040000

In [1061]:
#df_final_reports.to_csv("final_reports_data.csv")

<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Argument usage per unique Report</h2>
</div>

In [1062]:
# Count how many reports mention each argument at least once
def count_reports_with_argument(argument, df):
    return df['argument_list'].apply(lambda x: argument in x).sum()

df_grouped_arguments['part_of_reports_absolute'] = df_grouped_arguments['Argument'].apply(
    lambda arg: count_reports_with_argument(arg, df_reports_arguments)
)


In [1063]:
total_amount_of_reports = 917

df_grouped_arguments = df_grouped_arguments.sort_values("part_of_reports_absolute",ascending=False)
df_grouped_arguments["part_of_reports_relative"] = df_grouped_arguments["part_of_reports_absolute"] / total_amount_of_reports

In [1064]:
df_grouped_arguments.head(40)

,index,Argument,positive,neutral,negative,total,part_of_reports_absolute,part_of_reports_relative
221,221,Revenue,2102,272,593,2967,745,0.812432
123,123,Growth,2621,100,404,3125,722,0.787350
157,157,Margins,788,90,538,1416,551,0.600872
159,159,Market Conditions,394,172,550,1116,542,0.591058
165,165,Market Share,938,47,189,1174,498,0.543075
138,138,Investments,615,140,169,924,428,0.466739
32,32,Cost Management,447,77,354,878,394,0.429662
20,20,Competition,141,60,591,792,380,0.414395
80,80,Earnings per Share,452,127,123,702,344,0.375136
73,73,Earnings,288,66,146,500,334,0.364231


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Industry Analysis</h2>
</div>

In [1065]:
list_of_industries = [
    "Communication Services",
    "Automobile",
    "Travel",
    "Materials",
    "Healthcare",
    "Consumer Discretionary",
    "Energy",
    "Technology",
    "Industrials",
    "Retail",
    "Consumer Staples",
    "Beauty",
    "Financials",
    "Clothing",
    "Food",
]

df_industry_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative","argument_importance","positive_ratio","negative_ratio","neutral_ratio"])

# Initialize columns for each industry
for industry in list_of_industries:
    df_industry_analysis[f'{industry}_absolute'] = 0
    df_industry_analysis[f'{industry}_relative'] = 0.0

# Count absolute and relative mentions per industry
for industry in list_of_industries:
    # Filter reports for the current industry
    industry_reports = df_final_reports[df_final_reports['industry'] == industry]
    total_reports_for_industry = len(industry_reports)

    if total_reports_for_industry > 0:  # Avoid division by zero
        df_industry_analysis[f'{industry}_absolute'] = df_industry_analysis['Argument'].apply(
            lambda arg: industry_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_industry_analysis[f'{industry}_relative'] = round((df_industry_analysis[f'{industry}_absolute'] / total_reports_for_industry),3)

KeyError: "['argument_importance', 'positive_ratio', 'negative_ratio', 'neutral_ratio'] not found in axis"

In [ ]:
df_industry_analysis

,index,Argument,total,opinion_score,Communication Services_absolute,Communication Services_relative,Automobile_absolute,Automobile_relative,Travel_absolute,Travel_relative,...,Consumer Staples_absolute,Consumer Staples_relative,Beauty_absolute,Beauty_relative,Financials_absolute,Financials_relative,Clothing_absolute,Clothing_relative,Food_absolute,Food_relative
295,295,Revenue,2670,0.50,63,0.955,28,0.609,39,0.975,...,44,0.898,23,0.767,84,0.875,56,0.757,61,0.924
168,168,Growth,2558,0.79,54,0.818,38,0.826,27,0.675,...,37,0.755,26,0.867,69,0.719,63,0.851,60,0.909
212,212,Margins,1317,0.17,37,0.561,27,0.587,24,0.600,...,33,0.673,25,0.833,35,0.365,61,0.824,51,0.773
215,215,Market Conditions,1116,-0.14,35,0.530,26,0.565,31,0.775,...,33,0.673,25,0.833,62,0.646,51,0.689,42,0.636
222,222,Market Share,1174,0.64,54,0.818,22,0.478,24,0.600,...,32,0.653,17,0.567,49,0.510,41,0.554,35,0.530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,189,Investor Concerns,1,-1.00,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
192,192,LNG Pricing,1,1.00,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
197,197,Licensing Revenue,1,-1.00,1,0.015,0,0.000,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
201,201,Litigation and Regulation,1,-1.00,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,1,0.010,0,0.000,0,0.000


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Rating Analysis</h2>
</div>

In [ ]:
list_of_ratings = [
    "buy",
    "overweight",
    "neutral",
    "underweight",
    "hold",
    "equal weight",
    "underperform",
    "none",
    "outperform",
    "sell"
]

df_rating_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative","argument_importance","positive_ratio","negative_ratio","neutral_ratio"])

# Initialize columns for each rating
for rating in list_of_ratings:
    df_rating_analysis[f'{rating}_absolute'] = 0
    df_rating_analysis[f'{rating}_relative'] = 0.0

# Count absolute and relative mentions per rating
for rating in list_of_ratings:
    # Filter reports for the current rating
    rating_reports = df_final_reports[df_final_reports['rating'] == rating]
    total_reports_for_rating = len(rating_reports)

    if total_reports_for_rating > 0:  # Avoid division by zero
        df_rating_analysis[f'{rating}_absolute'] = df_rating_analysis['Argument'].apply(
            lambda arg: rating_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_rating_analysis[f'{rating}_relative'] = round((df_rating_analysis[f'{rating}_absolute'] / total_reports_for_rating),3)

In [ ]:
df_rating_analysis

,index,Argument,total,opinion_score,buy_absolute,buy_relative,overweight_absolute,overweight_relative,neutral_absolute,neutral_relative,...,equal weight_absolute,equal weight_relative,underperform_absolute,underperform_relative,none_absolute,none_relative,outperform_absolute,outperform_relative,sell_absolute,sell_relative
295,295,Revenue,2670,0.50,206,0.877,245,0.795,69,0.852,...,56,0.767,4,0.364,9,0.529,3,1.000,7,1.000
168,168,Growth,2558,0.79,197,0.838,256,0.831,55,0.679,...,49,0.671,11,1.000,7,0.412,2,0.667,5,0.714
212,212,Margins,1317,0.17,139,0.591,188,0.610,41,0.506,...,46,0.630,7,0.636,2,0.118,3,1.000,4,0.571
215,215,Market Conditions,1116,-0.14,133,0.566,202,0.656,49,0.605,...,47,0.644,1,0.091,14,0.824,1,0.333,3,0.429
222,222,Market Share,1174,0.64,142,0.604,183,0.594,34,0.420,...,42,0.575,0,0.000,7,0.412,1,0.333,6,0.857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,189,Investor Concerns,1,-1.00,0,0.000,1,0.003,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
192,192,LNG Pricing,1,1.00,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
197,197,Licensing Revenue,1,-1.00,0,0.000,0,0.000,1,0.012,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000
201,201,Litigation and Regulation,1,-1.00,0,0.000,1,0.003,0,0.000,...,0,0.000,0,0.000,0,0.000,0,0.000,0,0.000


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Year Analysis</h2>
</div>

In [ ]:
df_year_analysis = df_grouped_arguments.drop(columns=["positive","negative","neutral","part_of_reports_absolute","part_of_reports_relative","argument_importance","positive_ratio","negative_ratio","neutral_ratio"])

In [ ]:
df_year_analysis

,index,Argument,total,opinion_score
295,295,Revenue,2670,0.50
168,168,Growth,2558,0.79
212,212,Margins,1317,0.17
215,215,Market Conditions,1116,-0.14
222,222,Market Share,1174,0.64
...,...,...,...,...
189,189,Investor Concerns,1,-1.00
192,192,LNG Pricing,1,1.00
197,197,Licensing Revenue,1,-1.00
201,201,Litigation and Regulation,1,-1.00


In [ ]:
years = list(range(2014, 2023))

# Initialize columns for each year
for year in years:
    df_year_analysis[f'{year}_absolute'] = 0
    df_year_analysis[f'{year}_relative'] = 0.0

# Count absolute and relative mentions per year
for year in years:
    # Filter reports for the current year
    year_reports = df_final_reports[df_final_reports['year'] == year]
    total_reports_for_year = len(year_reports)

    if total_reports_for_year > 0:  # Avoid division by zero
        df_year_analysis[f'{year}_absolute'] = df_year_analysis['Argument'].apply(
            lambda arg: year_reports['argument_list'].apply(lambda x: arg in x).sum()
        )
        df_year_analysis[f'{year}_relative'] = round((df_year_analysis[f'{year}_absolute'] / total_reports_for_year),3)

In [ ]:
# Calculate standard deviation and variance for the relative values
relative_columns = [f'{year}_relative' for year in range(2014, 2023)]

df_year_analysis['relative_std_dev'] = df_year_analysis[relative_columns].std(axis=1)
df_year_analysis['relative_variance'] = df_year_analysis[relative_columns].var(axis=1)

In [ ]:
df_year_analysis.head(40)

,index,Argument,total,opinion_score,2014_absolute,2014_relative,2015_absolute,2015_relative,2016_absolute,2016_relative,...,2019_absolute,2019_relative,2020_absolute,2020_relative,2021_absolute,2021_relative,2022_absolute,2022_relative,relative_std_dev,relative_variance
295,295,Revenue,2670,0.50,46,0.836,44,0.815,38,0.905,...,49,0.860,156,0.848,146,0.793,173,0.776,0.044988,0.002024
168,168,Growth,2558,0.79,51,0.927,50,0.926,38,0.905,...,48,0.842,141,0.766,147,0.799,152,0.682,0.082219,0.006760
212,212,Margins,1317,0.17,38,0.691,28,0.519,21,0.500,...,32,0.561,116,0.630,108,0.587,136,0.610,0.058644,0.003439
215,215,Market Conditions,1116,-0.14,39,0.709,26,0.481,23,0.548,...,23,0.404,127,0.690,99,0.538,150,0.673,0.111276,0.012382
222,222,Market Share,1174,0.64,37,0.673,33,0.611,29,0.690,...,29,0.509,98,0.533,96,0.522,111,0.498,0.071214,0.005071
188,188,Investments,864,0.53,33,0.600,30,0.556,17,0.405,...,30,0.526,76,0.413,79,0.429,108,0.484,0.066509,0.004423
44,44,Cost Management,778,0.17,26,0.473,18,0.333,14,0.333,...,22,0.386,90,0.489,85,0.462,94,0.422,0.059551,0.003546
31,31,Competition,724,-0.66,40,0.727,23,0.426,22,0.524,...,26,0.456,76,0.413,64,0.348,63,0.283,0.136537,0.018642
112,112,Earnings per Share,689,0.46,37,0.673,36,0.667,34,0.810,...,25,0.439,64,0.348,53,0.288,34,0.152,0.208826,0.043608
102,102,Earnings,500,0.29,4,0.073,18,0.333,10,0.238,...,24,0.421,80,0.435,72,0.391,98,0.439,0.126431,0.015985


<div style="background-color: #181818; padding: 5px; border-radius: 20px; text-align: center;">
    <h2 style="font-size: 1.7em;color:#FBFAFA">Target Price Analysis</h2>
</div>

df_reports

In [ ]:
df_reports["difference_tp_start_price"] = df_reports["adjusted_target_price"] - df_reports["start price"]
df_reports["difference_tp_start_price_percentage"] = (df_reports["adjusted_target_price"] - df_reports["start price"]).abs() / df_reports["start price"] * 100

In [ ]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   document_id                           916 non-null    int64  
 1   filename                              916 non-null    object 
 2   date                                  916 non-null    object 
 3   provider                              916 non-null    object 
 4   ticker                                916 non-null    object 
 5   company_name                          916 non-null    object 
 6   industry                              916 non-null    object 
 7   target_price                          705 non-null    float64
 8   rating                                916 non-null    object 
 9   start price                           916 non-null    float64
 10  one day after                         916 non-null    float64
 11  max price after 3 m

In [ ]:
df_grouped_rating = df_reports[df_reports['adjusted_target_price'] > 0]

df_grouped_rating = df_grouped_rating.groupby('rating', as_index=False).agg(
    total_reports=("filename", 'count'),
    average_difference=("difference_tp_start_price_percentage", 'mean'),
)


In [ ]:
df_grouped_rating

,rating,total_reports,average_difference
0,buy,228,34.581394
1,equal weight,73,15.322290
2,hold,57,13.772914
3,neutral,34,14.707363
4,none,1,10.344828
5,outperform,3,19.431983
6,overweight,276,22.621519
7,sell,7,15.418232
8,underperform,1,51.876805
9,underweight,25,29.105807
